# **Create a Map to Visualize Weather Data**

In [ ]:
import geopandas as gpd
import contextily as ctx

from skimage import io
import matplotlib.pyplot as plt
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

In [ ]:
def add_icon(row):
    img = io.imread(f"https://openweathermap.org/img/wn/{row.icon}@2x.png")
    img_offset = OffsetImage(img, zoom=0.4, alpha=1, )

    ab = AnnotationBbox(img_offset,[
        row.geometry.x+150000, 
        row.geometry.y-110000
    ], frameon=False)

def GeoAXIS_PLOT(geo_dataframe: gpd.GeoDataFrame):
    ax = geo_dataframe.to_crs(epsg=3857).plot(figsize=(15, 8), color="black")

    geo_dataframe.to_crs(epsg=3857).apply(add_icon, axis=1)

    geo_dataframe.to_crs(epsg=3857).apply(lambda x: ax.annotate(text=f"{x.city}  ", 
                                                  fontsize=10, color="black", 
                                                  xy=x.geometry.centroid.coords[0], 
                                                  ha='right'), axis=1);

    geo_dataframe.to_crs(epsg=3857).apply(lambda x: ax.annotate(text=f" {round(x.temp)}°", 
                                                  fontsize=15, 
                                                  color="black", 
                                                  xy=x.geometry.centroid.coords[0], 
                                                  ha='left'), axis=1);

    xmin, ymin, xmax, ymax = geo_dataframe.to_crs(epsg=3857).total_bounds
    margin_y = .2
    margin_x = .2
    y_margin = (ymax - ymin) * margin_y
    x_margin = (xmax - xmin) * margin_x

    ax.set_xlim(xmin - x_margin, xmax + x_margin)
    ax.set_ylim(ymin - y_margin, ymax + y_margin)

    ctx.add_basemap(ax, source=ctx.providers.OpenStreetMap.Mapnik)

    ax.set_axis_off()